In [1]:
import regex as re
import pandas as pd
import pdfplumber

In [2]:
pdf = pdfplumber.open('DataUUHabibie/uu23-1999.pdf')
page = pdf.pages[:]
text = ''

text += page[0].extract_text()

for i in page[1:]:
    text += "\n" + i.extract_text()

file = text

In [3]:
# file_path = "uu7-1999.txt"

# try:
#     with open(file_path, encoding="utf8") as file:
#         file = file.read()
# except:
#     with open(file_path) as file:
#         file = file.read()

## Remove Header

In [4]:
file_no_header = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file, flags=re.DOTALL)
file_no_header = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", file_no_header, flags=re.DOTALL)

In [5]:
file_no_header

'PRESIDEN\nREPUBLIK INDONESIA\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 23 TAHUN 1999\nTENTANG\nBANK INDONESIA\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nPRESIDEN REPUBLIK INDONESIA,\nMenimbang : a. bahwa untuk memelihara kesinambungan pelaksanaan pembangunan\nnasional guna mewujudkan masyarakat Indonesia yang adil dan\nmakmur berdasarkan Pancasila dan Undang-Undang Dasar 1945,\npelaksanaan pembangunan ekonomi diarahkan kepada terwujudnya\nperekonomian nasional yang berpihak pada ekonomi kerakyatan,\nmerata, mandiri, andal, berkeadilan dam mampu bersaing di kancah\nperekonomian internasional;\nb. bahwa guna mendukung terwujudnya perekonomian nasional\nsebagaimana tersebut di atas dan sejalan dengan tantangan\nperkembangan dan pembangunan ekonomi yang semakin kompleks,\nsistem keuangan yang semakin maju serta perekonomian\ninternasional yang semakin kompetitif dan terintegrasi, kebijakan\nmoneter harus dititikberatkan pada upaya untuk memelihara\nstabilitas nilai rupiah;\nc. bahwa untuk menetap

## Pisah Penjelasan

In [6]:
penjelasan = re.search("(Agar setiap orang mengetahuinya.*|Agar setiap orang dapat mengetahuinya.*)", file_no_header, re.DOTALL).group()
badan = file_no_header.replace(penjelasan, "")

In [7]:
penjelasan = re.sub("TAMBAHAN LEMBARAN NEGARA.*", "", penjelasan, flags=re.DOTALL)

# Badan

## Ambil BAB

In [8]:
def ambil_bab(text):
    pattern = "(?<=\\n)(BAB\s*[IVXLCDM]+.*?)(?=\\nBAB|$)"
    bab = re.findall(pattern, text, re.DOTALL)
    if bab:
        ## ambil nomor dan nama bab
        pattern_no = "(?<=BAB)(?:\s*)([IVXLCDM]+)"
        pattern_nama = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)(.*?)(?=\\n\p{Lu}\p{Ll})"
        pattern_isi = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n.*?)(?:\\n)(\p{Lu}\p{Ll}.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in bab]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bab]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bab]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n)(BAB [IVXLCDM]+.*?)(?=\\nBAB|$)<br>
Digunakan untuk mengambil semua bab<br>
Kondisi:<br>
    a. Text dimulai dengan \n tanpa mengambil \n<br>
    b. Setelah \n dimulai dengan BAB(spasi)[Bilangan Romawi 1 digit dapat diulang beberapa kali] contoh (BAB IIV, BAB VVIX) akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBAB<br>

2. (?<=BAB)(?:\s*)([IVXLCDM]+)<br>
Digunakan untuk mengambil no bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas [Bilangan romawi 1 digit dapat diulang beberapa kali] akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>

3. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)([\p{Lu}\s\\n]*)(?=\\n)<br>
Digunakan untuk mengambil nama bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Mengambil semua karakter berupa huruf kapital atau spasi atau \n<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \n<br>

4. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n[\p{Lu}\s\\n]*)(?:\\n)(.*)<br>
Digunakan untuk mengambil isi bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Setelah kondisi di atas teks dimulai dengan huruf kapital atau spasi atau \n hingga berhenti pada \n tanpa diambil (nama bab)<br>
    c. Semua karakter sesudahnya diambil<br>

## Ambil Bagian

In [9]:
def ambil_bagian(text):
    pattern = "(?<=\\n|^)(Bagian\s*\p{Lu}.*?)(?=\\nBagian\s*\p{Lu}|$)"
    bagian = re.findall(pattern, text, re.DOTALL)
    if bagian:
        pattern_no = "(?<=Bagian)(?:\s*)(\p{Lu}\p{L}*\s*)(?=\\n)"
        pattern_nama = "(?<=Bagian)(?:\s*\p{Lu}\p{L}*\s*\\n)(.*?)(?=\\n(Pasal|Paragraf))"
        pattern_isi = "Pasal.*"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in bagian]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bagian]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bagian]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n|^)(Bagian \p{Lu}.*?)(?=\\nBagian \p{Lu}|$)<br>
Digunakan untuk mengambil semua bagian<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Bagian(spasi)[Huruf kapital] contoh (Bagian H, Bagian C, Bagian D) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBagian(spasi)[Huruf kapital]<br>

2. (?<=Bagian \p{Lu})(?:\p{L}*\s*\\n)(.*?)(?=\\nPasal)<br>
Digunakan untuk mengambil nama bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Bagian(spasi)[Huruf kapital][Huruf atau spasi dapat diulang beberapa kali]\n tanpa diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \nPasal<br>

3. Pasal.*<br>
Digunakan untuk mengambil isi bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal dengan diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>

## Ambil Pasal

In [10]:
def ambil_pasal(text):
    pattern = "(?<=\\n|^)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal\s*[\p{N}IVXLCDM]+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(Pasal \p{N}+\s*\\n.*?)(?=\\nPasal|$)<br>
Digunakan untuk mengambil semua pasal<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Pasal(spasi)[Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n contoh (Pasal 1\n, Pasal 23     \n) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. Pengambilan karakter akan berhenti ketika bertemu \nPasal atau (end of string)<br>

2. (?<=Pasal)(?:\s*)(\p{N}+)<br>
Digunakan untuk mengambil no pasal<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali] akan diambil<br>

3. (?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)<br>
Digunakan untuk mengambil isi pasal<br>
Kondisi:<br>
    a. Teks dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n akan diambil (nomor pasal)<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>

## Ambil Ayat

In [11]:
def ambil_ayat(text):
    clean_text = re.sub('(?<=ayat)(\\n)(?=\(\p{N}\))', "", text, flags=re.DOTALL)
    ayat = re.findall('(?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)', clean_text, flags=re.DOTALL)
    
    if ayat:
        pattern_no = "(?<=\\n|^)(?:\()(\p{N}+)(?=\))"
        pattern_isi = "(?<=\(\p{N}+\)\s*)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)<br>
Digunakan untuk mengambil semua ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung)<br>
    c. Setelah kondisi diatas semua karakter diambil<br>
    d. Pengambilan karakter diambil hingga bertemu \n(buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) atau (end of string)<br>

2. (?<=\\n|^)(?:\()(\p{N}+)(?=\))<br>
Digunakan untuk mengambil no ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung) tanpa diambil<br>
    c. Setelah kondisi diatas dimulai oleh [nomor dapat diulang beberapa kali] diambil<br>
    d. Pengambilan karakter diambil hingga bertemu (tutup kurung)<br>

3. (?<=\(\p{N}+\)\s*)(.*)<br>
Digunakan untuk mengambil isi ayat<br>
Kondisi:<br>
    a. Text dimulai dengan (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) tanpa diambil (no ayat)<br>
    b. Setelah kondisi diatas semua karakter diambil<br>

# Penjelasan

In [12]:
penjelasan

'Agar setiap orang mengetahuinya, memerintahkan pengundangan\nundang-undang ini dengan penempatannya dalam Lembaran Negara \nRepublik Indonesia.\nDisahkan di Jakarta\npada tanggal 17 Mei 1999\nPRESIDEN REPUBLIK INDONESIA\nttd\nBACHRUDDIN JUSUF HABIBIE\nDiundangkan di jakarta\npada tanggal 17 Mei 1999\nMENTERI NEGARA SEKRETARIS NEGARA\nREPUBLIK INDONESIA\nttd\nPROF. DR. H. MULADI, SH\nLAMBARAN NEGARA REPUBLIK INDONESIA TAHUN 1999 NOMOR 66\nPRESIDEN\nREPUBLIK INDONESIA\nPENJELASAN\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 23 TAHUN 1999\nTENTANG\nBANK INDONESIA\nUMUM\nPembangunan nasional Indonesia untuk mewujudkan suatu masyarakat adil dan makmur\nberdasarkan Pancasila dan Undang-Undang Dasar 1945 telah mencapai berbagai\nkemajuan termasuk di bidang ekonomi dan moneter, sebagaimana tercermin pada\npertumbuhan ekonomi yang cukup tinggi dan tingkat inflasi yang terkendali. Sementara\nitu, dalam pembangunan tersebut terdapat kelemahan struktur dan sistem perekonomian\nIndonesia yang minimbul

## Ambil Pasal

In [13]:
def ambil_penjelasan_pasal(text):
    pattern = "(?<=\\n|^|\s)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal \p{N}+\s*\\n|\sPasal\s*[\p{N}IVXLCDM]+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

## Ambil Ayat

In [14]:
def ambil_penjelasan_ayat(text):
    pattern = "(?<=\\n|^|\s)(Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|\sAyat\s*\(\p{N}+\)|$)"
    ayat = re.findall(pattern, text, re.DOTALL)
    if ayat:
        pattern_no = "(?<=Ayat)(?:\s*\()(\p{N}+)"
        pattern_isi = "(?<=Ayat)(?:\s*\(\p{N}+\)\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [15]:
df_badan_lst = []
lst_no_bab, lst_nama_bab, lst_isi_bab = ambil_bab(badan)

for i_bab, no_bab in enumerate(lst_no_bab):
    nama_bab = lst_nama_bab[i_bab]
    lst_no_bagian, lst_nama_bagian, lst_isi_bagian = ambil_bagian(lst_isi_bab[i_bab])

    for i_bagian, no_bagian in enumerate(lst_no_bagian):
        nama_bagian = lst_nama_bagian[i_bagian]
        lst_no_pasal, lst_isi_pasal = ambil_pasal(lst_isi_bagian[i_bagian])

        for i_pasal, no_pasal in enumerate(lst_no_pasal):
            lst_no_ayat, lst_isi_ayat = ambil_ayat(lst_isi_pasal[i_pasal])
            df_ayat = pd.DataFrame()
            df_ayat["no_ayat"] = lst_no_ayat
            df_ayat["isi_ayat"] = lst_isi_ayat
            df_ayat["no_bab"] = no_bab
            df_ayat["nama_bab"] = nama_bab
            df_ayat["no_bagian"] = no_bagian
            df_ayat["nama_bagian"] = nama_bagian
            df_ayat["no_pasal"] = no_pasal
            df_badan_lst.append(df_ayat)

df_badan = pd.concat(df_badan_lst)
df_badan = df_badan[["no_bab", "nama_bab", "no_bagian", "nama_bagian", "no_pasal", "no_ayat", "isi_ayat"]]

In [16]:
df_badan

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat
0,I,KETENTUAN UMUM,,,1,,Dalam Undang-undang ini yang dimaksudkan denga...
0,I,KETENTUAN UMUM,,,2,1,Satuan mata uang negara Republik Indonesia ad...
1,I,KETENTUAN UMUM,,,2,2,Uang rupiah adalah alat pembayaran yang sah d...
2,I,KETENTUAN UMUM,,,2,3,Setiap perbuatan yang menggunakan uang atau m...
3,I,KETENTUAN UMUM,,,2,4,Setiap orang atau badan yang berada di wilaya...
...,...,...,...,...,...,...,...
2,XVII,KETENTUAN PERALIHAN,,,76,3,Dalam hal diperlukan perpanjangan jangka wakt...
0,XVII,KETENTUAN PERALIHAN,,,77,,Dalam jangka waktu paling lama 2 (dua) tahun s...
0,XVII,KETENTUAN PERALIHAN,,,78,1,"Dengan berlakunya Undang-undang ini, Undang-u..."
1,XVII,KETENTUAN PERALIHAN,,,78,2,Peraturan pelaksanaan dari undang-undang Nomo...


In [17]:
df_penjelasan_lst = []

lst_no_penjelasan_pasal, lst_isi_penjelasan_pasal = ambil_penjelasan_pasal(penjelasan)

for i_penjelasan_pasal, no_penjelasan_pasal in enumerate(lst_no_penjelasan_pasal):
    lst_no_penjelasan_ayat, lst_isi_penjelasan_ayat = ambil_penjelasan_ayat(lst_isi_penjelasan_pasal[i_penjelasan_pasal])
    df_penjelasan_ayat = pd.DataFrame()
    df_penjelasan_ayat['penjelasan'] = lst_isi_penjelasan_ayat
    df_penjelasan_ayat['no_ayat'] = lst_no_penjelasan_ayat
    df_penjelasan_ayat['no_pasal'] = no_penjelasan_pasal
    df_penjelasan_lst.append(df_penjelasan_ayat)

df_penjelasan = pd.concat(df_penjelasan_lst)
df_penjelasan = df_penjelasan[['no_pasal', 'no_ayat', 'penjelasan']]

In [18]:
df_penjelasan

,no_pasal,no_ayat,penjelasan
0,1,,Cukup jelas
0,2,1,Satu rupiah terdiri atas 100 (seratus) sen.
1,2,2,Yang dimaksud dengan wilayah negara Republik I...
2,2,3,Pokok-pokok ketentuan mengenai pengecualian se...
3,2,4,Dalam hal terdapat keraguan atas keaslian uang...
...,...,...,...
2,76,3,Yang dimaksud dengan hari pada ayat ini adalah...
0,77,,Cukup jelas
0,78,1,Cukup jelas
1,78,2,Cukup jelas


In [19]:
penjelasan

'Agar setiap orang mengetahuinya, memerintahkan pengundangan\nundang-undang ini dengan penempatannya dalam Lembaran Negara \nRepublik Indonesia.\nDisahkan di Jakarta\npada tanggal 17 Mei 1999\nPRESIDEN REPUBLIK INDONESIA\nttd\nBACHRUDDIN JUSUF HABIBIE\nDiundangkan di jakarta\npada tanggal 17 Mei 1999\nMENTERI NEGARA SEKRETARIS NEGARA\nREPUBLIK INDONESIA\nttd\nPROF. DR. H. MULADI, SH\nLAMBARAN NEGARA REPUBLIK INDONESIA TAHUN 1999 NOMOR 66\nPRESIDEN\nREPUBLIK INDONESIA\nPENJELASAN\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 23 TAHUN 1999\nTENTANG\nBANK INDONESIA\nUMUM\nPembangunan nasional Indonesia untuk mewujudkan suatu masyarakat adil dan makmur\nberdasarkan Pancasila dan Undang-Undang Dasar 1945 telah mencapai berbagai\nkemajuan termasuk di bidang ekonomi dan moneter, sebagaimana tercermin pada\npertumbuhan ekonomi yang cukup tinggi dan tingkat inflasi yang terkendali. Sementara\nitu, dalam pembangunan tersebut terdapat kelemahan struktur dan sistem perekonomian\nIndonesia yang minimbul

In [20]:
df_penjelasan['pasal_ayat'] = df_penjelasan['no_pasal'] + "-" + df_penjelasan['no_ayat']
df_badan['pasal_ayat'] = df_badan['no_pasal'] + "-" + df_badan['no_ayat']

In [21]:
df_left = df_badan.merge(df_penjelasan, how="left", on="pasal_ayat", suffixes=('', '_y'))
df_left = df_left.drop(['pasal_ayat'], axis=1)

In [22]:
df_left[pd.isna(df_left['penjelasan'])]

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,no_pasal_y,no_ayat_y,penjelasan
9,II,"STATUS, TEMPAT KEDUDUKAN, DAN MODAL",,,4,2,Bank Indonesia adalah lembaga negara yang ind...,NaN,NaN,NaN
10,II,"STATUS, TEMPAT KEDUDUKAN, DAN MODAL",,,4,3,Bank Indonesia adalah badan hukum berdasarkan...,NaN,NaN,NaN
67,VI,TUGAS MENGATUR DAN MENGAWASI BANK,,,30,1,Bank Indonesia dapat menugasi pihak lain untu...,NaN,NaN,NaN
161,XI,KETENTUAN PIDANA DAN SANKSI ADMINISTRATIF,,,71,2,Apabila pelanggaran sebagaimana dimaksud pada...,NaN,NaN,NaN
162,XI,KETENTUAN PIDANA DAN SANKSI ADMINISTRATIF,,,71,3,Keterangan dan data lainnya yang bersifat rah...,NaN,NaN,NaN
164,XI,KETENTUAN PIDANA DAN SANKSI ADMINISTRATIF,,,72,2,Sanksi administratif sebagaimana dimaksud pad...,NaN,NaN,NaN
165,XI,KETENTUAN PIDANA DAN SANKSI ADMINISTRATIF,,,72,3,Ketentuan lebih lanjut mengenai sanksi admini...,NaN,NaN,NaN


In [23]:
df_left[df_left['no_pasal_y'] == '35']

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,no_pasal_y,no_ayat_y,penjelasan


In [24]:
df_right = df_badan.merge(df_penjelasan, how="right", on="pasal_ayat", suffixes=('', '_y'))
df_right = df_right.drop(['pasal_ayat'], axis=1)

In [25]:
df_right[pd.isna(df_right['isi_ayat'])]

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,no_pasal_y,no_ayat_y,penjelasan
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,,Cukup jelas
74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,,Cukup jelas
158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,3,Pokok-pokok ketentuan yang akan ditetapkan dal...


In [26]:
df_right[df_right['no_pasal_y'] == '17'].loc[158]['penjelasan']

'Pokok-pokok ketentuan yang akan ditetapkan dalam Peratuan Dewan Gubernur\nmemuat antara lain :\na. jenis keterangan dan data lainnya yang dikategorikan rahasia, antara lain\nketerangan dan data individual yang diperoleh melalui survei dan data\nindividual Bank peseta kliring;\nb. perlakuan terhadap keterangan dan data lainnya yang bersifat rahasia;\nc. prosedur pengungkapan keterangan dan data lainnya yang bersifat rahasia;\nd. pejabat yang berwenang mengungkapkan keterangan dan data lainnya yang\nbersifat rahasia.'

In [27]:
df = df_badan.merge(df_penjelasan, how="inner", on="pasal_ayat", suffixes=('', '_y'))
df = df.drop(['pasal_ayat', 'no_pasal_y', 'no_ayat_y'], axis=1)

In [28]:
df

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,penjelasan
0,I,KETENTUAN UMUM,,,1,,Dalam Undang-undang ini yang dimaksudkan denga...,Cukup jelas
1,I,KETENTUAN UMUM,,,2,1,Satuan mata uang negara Republik Indonesia ad...,Satu rupiah terdiri atas 100 (seratus) sen.
2,I,KETENTUAN UMUM,,,2,2,Uang rupiah adalah alat pembayaran yang sah d...,Yang dimaksud dengan wilayah negara Republik I...
3,I,KETENTUAN UMUM,,,2,3,Setiap perbuatan yang menggunakan uang atau m...,Pokok-pokok ketentuan mengenai pengecualian se...
4,I,KETENTUAN UMUM,,,2,4,Setiap orang atau badan yang berada di wilaya...,Dalam hal terdapat keraguan atas keaslian uang...
...,...,...,...,...,...,...,...,...
168,XVII,KETENTUAN PERALIHAN,,,76,3,Dalam hal diperlukan perpanjangan jangka wakt...,Yang dimaksud dengan hari pada ayat ini adalah...
169,XVII,KETENTUAN PERALIHAN,,,77,,Dalam jangka waktu paling lama 2 (dua) tahun s...,Cukup jelas
170,XVII,KETENTUAN PERALIHAN,,,78,1,"Dengan berlakunya Undang-undang ini, Undang-u...",Cukup jelas
171,XVII,KETENTUAN PERALIHAN,,,78,2,Peraturan pelaksanaan dari undang-undang Nomo...,Cukup jelas


## UU YANG SUDAH BISA
UU1-1999<br>
UU6-1999 (Pasalnya pake romawi) FIXED<br>
UU7-1999 (Gaada penjelasan) FIXED<br>
UU8-1999<br>
UU9-1999<br>
UU10-1999<br>
UU11-1999<br>
UU13-1999<br>
UU19-1999<br>
UU21-1999<br>

## UU YANG BELOM BISA
UU12-1999 (Penjelasan gaada ayat)<br>
UU15-1999 (Nomor ayat salah, penjelasan untuk 2 ayat kurung 2 kali)<br>
UU16-1999 (Badan gaada ayat)<br>
UU20-1999 (Penjelasan pasal ada titiknya)<br>
UU22-1999 (Penjelasan gaada ayat)<br>